Downloading training dataset MS1M-ArcFace (85K ids/5.8M images) and unzip

In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1SXS4-Am3bsKSK615qbYdbA_FMVh3sAvR' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1SXS4-Am3bsKSK615qbYdbA_FMVh3sAvR" -O faces_emore.zip && rm -rf /tmp/cookies.txt

In [ ]:
!unzip faces_emore.zip

Get evaluation data and extract

In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1HBGwyTFnl4Bt4hl5BpLE3t__J84R72TX' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1HBGwyTFnl4Bt4hl5BpLE3t__J84R72TX" -O faces_emore.zip && rm -rf /tmp/cookies.txt

In [ ]:
!tar MagFace-main/eval/eval_recognition -xvf lfw_cfp_agedb.tar

Extract data training data to /dataset/data (stop when needed)

In [ ]:
%python python rec2image.py --include ./dataset/faces_emore --output ./dataset/data

Install depenencies

In [ ]:
# Dependencies needed for Colab
colab_Dependencies = ["torchshard", "loguru", "sklearn", "logger", "mxnet-cu101", "mxnet"]
for lib in colab_Dependencies:
  print(f"Installing {lib}\n")
  %pip install {lib}
  print(f"\n{lib} installed!\n")

In [ ]:
# All dependencies 

requirments_list = ["torchvision", "torchshard", "torch", "numpy", "scipy", "termcolor", "opencv-contrib-python", "jupyterlab", "notebook", "seaborn", "Pillow==6.2.2", "matplotlib ", "loguru", "six", "imageio", "scikit-image", "tqdm", "sklearn"]
for lib in requirments_list:
  print(f"Installing {lib}\n")
  %pip install {lib}
  print(f"\n{lib} installed!\n")


Align faces

In [ ]:
%python face_align.py

In [ ]:
import os
dataset_Path = '/datasets/data'
dataset_Folders = os.listdir(dataset_Path)
print(sorted(dataset_Folders))
# list of values to remove
remove_list = ['faces_emore', 'train.list', '.train.list.swp']
for rm in remove_list:
  if rm in dataset_Folders:
    dataset_Folders.remove(rm)
print(sorted(dataset_Folders))

If needed, remove empty folders from training set (Case where image extraction ended early)

In [ ]:
# Create list of id's
# Remove id's with no images
print("Folders with 0 files")
for folder in dataset_Folders:
  length = len(os.listdir(f"{dataset_Path}/{folder}"))
  if length == 0:
    print(f"Removing {folder}")
    path = f"{dataset_Path}/{folder}"
    %rmdir {path}
# Confirm list
emptyFolderList = []
dataset_Folders = os.listdir(dataset_Path)
# Remove permenent folder / files
for rm in remove_list:
  if rm in dataset_Folders:
    dataset_Folders.remove(rm)

for folder in dataset_Folders:
  length = len(os.listdir(f"{dataset_Path}/{folder}"))
  if length == 0:
    emptyFolderList.append(folder)

if len(emptyFolderList) == 0:
  print("No empty folders")

Create a train.list file to train with

In [ ]:
# Write list to train.list
dataset_Folders = os.listdir(dataset_Path)
# Create train.list file
f = open(f"{dataset_Path}/train.list", "w")
# Write list
for idx, folder in enumerate(dataset_Folders):
  folder_Path = (f"{dataset_Path}/{folder}")
  img_List = os.listdir(f"{folder_Path}")
  for img in img_List:
    f.write(f"{folder_Path}/{img} 0 {idx} 0\n")

f.close()

dataset_Path = '/content/drive/MyDrive/datasets'
#open and read the file after the appending:
f = open(f"{dataset_Path}/train.list", "r")
# Checking for number of images
x = len(f.readlines())
print('Total lines:', x)

Train data

In [ ]:
# for default values
%%shell
cd /run
bash ./run_dist.sh

In [ ]:
# with gaussian filter
%%shell
cd /run
bash ./run_dist.sh --gaus True

If using WAVE adjust the "job-name" and "mail-user" values in the "./run/run_dist.sh" file, use the following commands in the terminal:

ssh login1 (fill in your)

module load PyTorch/1.12.1

module load TensorFlow/2.11.0-20230208

cd /run

<!-- Default training -->

sbatch -p gpu -N1 -n1 -c2 --gres=gpu:1 --time=08:00:00 --mem=32G run_dist.sh

<!-- with gaussian fitler -->
sbatch -p gpu -N1 -n1 -c2 --gres=gpu:1 --time=08:00:00 --mem=32G run_dist.sh --gaus True

Next is to evalute the trained model should output accuracy ratings for each dataset

In [ ]:
%%shell
cd /eval/eval_recognition
./eval.sh /run/test/00025.pth official 100

If using WAVE

ssh login1 (fill in your)

module load PyTorch/1.12.1

module load TensorFlow/2.11.0-20230208

cd /eval/eval_recognition

sbatch -p gpu -N1 -n1 -c2 --gres=gpu:1 --time=08:00:00 --mem=32G ./eval.sh /run/test/00025.pth official 100

Graphing data

In [ ]:
def storeOutput(values):
  epoch0, epoch1, epoch2, epoch3, epoch4, epoch5, epoch6 = [], [], [], [], [], [], []
  epochs = [epoch0, epoch1, epoch2, epoch3, epoch4, epoch5, epoch6]
  epochCounter = 0
  for epochCounter in range(7):
    for idx, line in enumerate(values):
      if (f'[{epochCounter}]') in line:
        # print(line)
        # Epoch 0
        if epochCounter == 0:
          epoch0.append(line)
        # Epoch 1
        if epochCounter == 1:
          epoch1.append(line)
        # Epoch 2
        if epochCounter == 2:
          epoch2.append(line)
        # Epoch 3
        if epochCounter == 3:
          epoch3.append(line)
        # Epoch 4
        if epochCounter == 4:
          epoch4.append(line)
        # Epoch 5
        if epochCounter == 5:
          epoch5.append(line)
        # Epoch 6
        if epochCounter == 6:
          epoch6.append(line)
        # Epoch 7
        if epochCounter == 7:
          epoch6.append(line)
  return(epochs)
  print('Epochs separeated')

In [ ]:
def getValues(epochs):
  # Loss
  ep0Loss, ep1Loss, ep2Loss, ep3Loss, ep4Loss, ep5Loss, ep6Loss = [], [], [], [], [], [], []
  epochLoss = [ep0Loss, ep1Loss, ep2Loss, ep3Loss, ep4Loss, ep5Loss, ep6Loss]
  # Accuracy
  ep0Acc, ep1Acc, ep2Acc, ep3Acc, ep4Acc, ep5Acc, ep6Acc = [], [], [], [], [], [], []
  epochAcc = [ep0Acc, ep1Acc, ep2Acc, ep3Acc, ep4Acc, ep5Acc, ep6Acc]
  # Normal
  ep0NormMean, ep1NormMean, ep2NormMean, ep3NormMean, ep4NormMean, ep5NormMean, ep6NormMean = [], [], [], [], [], [], []
  ep0NormMeanMin, ep1NormMeanMin, ep2NormMeanMin, ep3NormMeanMin, ep4NormMeanMin, ep5NormMeanMin, ep6NormMeanMin = [], [], [], [], [], [], []
  ep0NormMeanMax, ep1NormMeanMax, ep2NormMeanMax, ep3NormMeanMax, ep4NormMeanMax, ep5NormMeanMax, ep6NormMeanMax = [], [], [], [], [], [], []
  epochNormal = [ep0NormMean, ep1NormMean, ep2NormMean, ep3NormMean, ep4NormMean, ep5NormMean, ep6NormMean]
  epochNormalMin = [ep0NormMeanMin, ep1NormMeanMin, ep2NormMeanMin, ep3NormMeanMin, ep4NormMeanMin, ep5NormMeanMin, ep6NormMeanMin]
  epochNormalMax = [ep0NormMeanMax, ep1NormMeanMax, ep2NormMeanMax, ep3NormMeanMax, ep4NormMeanMax, ep5NormMeanMax, ep6NormMeanMax]
  # Margin
  ep0MargMean, ep1MargMean, ep2MargMean, ep3MargMean, ep4MargMean, ep5MargMean, ep6MargMean = [], [], [], [], [], [], []
  ep0MargMeanMin, ep1MargMeanMin, ep2MargMeanMin, ep3MargMeanMin, ep4MargMeanMin, ep5MargMeanMin, ep6MargMeanMin = [], [], [], [], [], [], []
  ep0MargMeanMax, ep1MargMeanMax, ep2MargMeanMax, ep3MargMeanMax, ep4MargMeanMax, ep5MargMeanMax, ep6MargMeanMax = [], [], [], [], [], [], []
  epochMargin = [ep0MargMean, ep1MargMean, ep2MargMean, ep3MargMean, ep4MargMean, ep5MargMean, ep6MargMean]
  epochMarginMin = [ep0MargMeanMin, ep1MargMeanMin, ep2MargMeanMin, ep3MargMeanMin, ep4MargMeanMin, ep5MargMeanMin, ep6MargMeanMin]
  epochMarginMax = [ep0MargMeanMax, ep1MargMeanMax, ep2MargMeanMax, ep3MargMeanMax, ep4MargMeanMax, ep5MargMeanMax, ep6MargMeanMax]

  for ep in epochs:
    for line in ep:
      if 'Epoch' in line:
        words = line.split('\t')
        firstLine = words[0].split(' ')
        # Epoch
        epochText = firstLine[12].replace(']', ' ')
        epochText = epochText.replace('[', ' ')
        epochText = epochText.split(" ")
        epVal = int(epochText[1])
        # Loss
        lossText = words[5].split(" ")
        loss = float(lossText[1])
        # Accuracy
        accText = words[8].split(" ")
        if accText[2] != '':
          acc = float(accText[2])
        else:
          acc = float(accText[3])
        # Epoch 0
        if epVal == 0:
          ep0Loss.append(loss)
          ep0Acc.append(acc)
        # Epoch 1
        if epVal == 1:
          ep1Loss.append(loss)
          ep1Acc.append(acc)
        # Epoch 2
        if epVal == 2:
          ep2Loss.append(loss)
          ep2Acc.append(acc)
        # Epoch 3
        if epVal == 3:
          ep3Loss.append(loss)
          ep3Acc.append(acc)
        # Epoch 4
        if epVal == 4:
          ep4Loss.append(loss)
          ep4Acc.append(acc)
        # Epoch 5
        if epVal == 5:
          ep5Loss.append(loss)
          ep5Acc.append(acc)
        # Epoch 6
        if epVal == 6:
          ep6Loss.append(loss)
          ep6Acc.append(acc)
      
          
      if 'debug info' in line:
        words = line.split(' ')
        debugEp = words[2]
        debugEp = debugEp.replace(']', '')
        debugEp = debugEp.replace('[', '')
        # for idx, i in enumerate(words):
        #   print(f"{idx}: {i}")
        if (('x_norm' in line)) and (debugEp == '0'):
          ep0NormMean.append(float(words[7]))
          ep0NormMeanMin.append(float(words[9]))
          ep0NormMeanMax.append(float(words[11]))
        if ('x_norm' in line) and (debugEp == '1'):
          ep1NormMean.append(float(words[7]))
          ep1NormMeanMin.append(float(words[9]))
          ep1NormMeanMax.append(float(words[11]))
        if ('x_norm' in line) and (debugEp == '2'):
          ep2NormMean.append(float(words[7]))
          ep2NormMeanMin.append(float(words[9]))
          ep2NormMeanMax.append(float(words[11]))
        if ('x_norm' in line) and (debugEp == '3'):
          ep3NormMean.append(float(words[7]))
          ep3NormMeanMin.append(float(words[9]))
          ep3NormMeanMax.append(float(words[11]))
        if ('x_norm' in line) and (debugEp == '4'):
          ep4NormMean.append(float(words[7]))
          ep4NormMeanMin.append(float(words[9]))
          ep4NormMeanMax.append(float(words[11]))
        if ('x_norm' in line) and (debugEp == '5'):
          ep5NormMean.append(float(words[7]))
          ep5NormMeanMin.append(float(words[9]))
          ep5NormMeanMax.append(float(words[11]))
        if ('x_norm' in line) and (debugEp == '6'):
          ep6NormMean.append(float(words[7]))
          ep6NormMeanMin.append(float(words[9]))
          ep6NormMeanMax.append(float(words[11]))

        if 'margin'in line and (debugEp == '0'):
          ep0MargMean.append(float(words[7]))
          ep0MargMeanMin.append(float(words[9]))
          ep0MargMeanMax.append(float(words[11]))
        if 'margin'in line and (debugEp == '1'):
          ep1MargMean.append(float(words[7]))
          ep1MargMeanMin.append(float(words[9]))
          ep1MargMeanMax.append(float(words[11]))
        if 'margin'in line and (debugEp == '2'):
          ep2MargMean.append(float(words[7]))
          ep2MargMeanMin.append(float(words[9]))
          ep2MargMeanMax.append(float(words[11]))
        if 'margin'in line and (debugEp == '3'):
          ep3MargMean.append(float(words[7]))
          ep3MargMeanMin.append(float(words[9]))
          ep3MargMeanMax.append(float(words[11]))
        if 'margin'in line and (debugEp == '4'):
          ep4MargMean.append(float(words[7]))
          ep4MargMeanMin.append(float(words[9]))
          ep4MargMeanMax.append(float(words[11]))
        if 'margin'in line and (debugEp == '5'):
          ep5MargMean.append(float(words[7]))
          ep5MargMeanMin.append(float(words[9]))
          ep5MargMeanMax.append(float(words[11]))
        if 'margin'in line and (debugEp == '6'):
          ep6MargMean.append(float(words[7]))
          ep6MargMeanMin.append(float(words[9]))
          ep6MargMeanMax.append(float(words[11]))
  print("Complete")
  return(epochLoss, epochAcc, epochNormal, epochNormalMin, epochNormalMax, epochMargin, epochMarginMin, epochMarginMax)
  

In [ ]:
def plotValues(loss, acc, norm, normMin, normMax, marg, margMin, margMax, label):
  import matplotlib.pyplot as plt

  for idx in range(7):

    plt.rcParams["figure.figsize"] = [14, 4]
    plt.rcParams["figure.autolayout"] = True

    plt.subplot(121)
    plt.plot(loss[idx])
    plt.title(f"Epoch {idx} Loss ({label})")

    plt.subplot(122)
    plt.plot(acc[idx])
    plt.title(f"Epoch {idx} Acc ({label})")

    plt.savefig(f'/output/{label}/lossAccEp{idx}.png')
    plt.show()

  for idx in range(7):

    plt.rcParams["figure.figsize"] = [14, 4]
    plt.rcParams["figure.autolayout"] = True

    plt.subplot(121)
    plt.plot(norm[idx], label="Normal")
    plt.plot(normMin[idx], label="Min")
    plt.plot(normMax[idx], label="Max")
    plt.title(f"Epoch {idx} Norm ({label})")

    plt.subplot(122)
    plt.plot(marg[idx], label="Margin")
    plt.plot(margMin[idx], label="Min")
    plt.plot(margMax[idx], label="Max")
    plt.title(f"Epoch {idx} Marg ({label})")

    plt.savefig(f'/output/{label}/normMarEp{idx}.png')
    plt.show()

In [ ]:
basePath = ['run/test/output.log', 'Base']
outputs = [basePath]
for outF, label in outputs:
  print(label)
  with open(outF) as my_file:
      outputLog = my_file.readlines()
  epochs = storeOutput(outputLog)
  loss, acc, norm, normMin, normMax, marg, margMin, margMax = getValues(epochs)
  plotValues(loss, acc, norm, normMin, normMax, marg, margMin, margMax, label)

Graph loss and accuracy over all epochs

In [ ]:
# loss, acc, norm, normMin, normMax, marg, margMin, margMax = getValues(epochs)
import matplotlib.pyplot as plt
L, A = [], []
fullLoss = [L, loss]
fullAcc = [A, acc]
full = [fullLoss, fullAcc]

for i in full:
  for ep in i[1]:
    for val in ep:
      i[0].append(val)

plt.plot(L)
plt.xlabel('1 per 100 Batchs (size 256)')
plt.ylabel('Loss')
plt.title('Loss')
  
# function to show the plot
plt.show()

print()
# plotting the points 
plt.plot(A)
plt.xlabel('1 per 100 Batchs (size 256)')
plt.ylabel('Accuracy')
plt.title('Accuracy')
  
# function to show the plot
plt.show()

Graphing normals and margins

In [ ]:
No, NMin, NMax= [], [], []
fullNorm = [No, norm]
fullNormMin = [NMin, normMin]
fullNormMax = [NMax, normMax]
fullNorms = [fullNorm, fullNormMin, fullNormMax]

for i in fullNorms:
  for ep in i[1]:
    for val in ep:
      i[0].append(val)

# plotting the points 
plt.plot(No)
plt.plot(NMin)
plt.plot(NMax)
plt.xlabel('1 per 100 Batchs (size 256)')
plt.ylabel('Norm')
# giving a title to my graph
plt.title('Normal')

In [ ]:
Ma, MMin, MMax= [], [], []
fullMarg = [Ma, marg]
fullMargMin = [MMin, margMin]
fullMargMax = [MMax, margMax]
fullMargs = [fullMarg, fullMargMin, fullMargMax]

for i in fullMargs:
  for ep in i[1]:
    for val in ep:
      i[0].append(val)

# plotting the points 
plt.plot(Ma)
plt.plot(MMin)
plt.plot(MMax)
plt.xlabel('1 per 100 Batchs (size 256)')
plt.ylabel('Marg')
# giving a title to my graph
plt.title('Margin')